### Jakub Zaręba

Celem projektu jest przewidywanie czasu okrążenia kierowcy F1 w wyścigu. Modele mają dostęp do następujących cech (10 cech), za pomocą których muszą podać czas okrążenia opisanego tymi cechami. Lista cech:
- identyfikator wyścigu (raceId)
- identyfikator kierowcy (driverId)
- numer aktualnego okrążenia, liczone od startu wyścigu 1 okrążenie=start (lap)
- aktualna pozycja kierowcy (position)
- najlepszy czas uzyskany w kwalifikacjach przez kierowcę (bestQualiTime)
- pozycja startowa kierowcy (grid)
- tego czy został wezwany na pitstop (isPitStop) -> jeżeli 1 oznacza to, że albo opony są już na skraju wytrzymałości lub doszło do uszkodzeń w bolidzie.
- identyfikator toru (circuitId)
- tego czy jest to okrążenie wyjazdowe z pitstopu (isOutLap) -> jeżeli 1, to kierowca w pewnym momencie alei serwisowej przekroczy linię mety będąc jeszcze w strefie ograniczenia prędkości oraz po wyjeździe jego opony wymagają "dogrzania", czas 1.  okrążenia po zjeździe na pewno będzie gorszy <br/><br/> Wykorzystane przeze mnie dane pochodzą ze strony kaggle.com [źródło](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020/). Jest to kompletny zbiór danych związany z Formułą 1, zawierający wszystkie możliwe informacje (wszystkie okrążenia, kierowcy, itd.) od 1950r. do 2024r.

### 1. Przygotowanie danych
Dane znajdują się w osobnych plikach, najpierw połączę je w jeden dataframe biblioteki pandas oraz pozbędę się niepotrzebnych dla modeli informacji. Przekształcę niektóre dane, by uzyskać dostęp do nowych cech, które mogą być przydatne dla modeli.

In [206]:
import pandas as pd

# Funkcja konwertująca czas w formacie X:XX.XXXX na minuty
def convert_time_to_min(time):
    try:
        time=time.replace(".",":").split(":")
        time_min=int(time[0])+int(time[1])/60+int(time[2])/60000
    except Exception as e:
        time_min=pd.NA
    return time_min

def read_data():
    lap_times=pd.read_csv("data/lap_times.csv")
    # Kolumna time zawiera czas w formacie X:XX.XXXX,
    # dane zawierają kolumnę milliseconds, która nie wymaga konwersji
    lap_times=lap_times.drop("time",axis=1)
    pit_stops=pd.read_csv("data/pit_stops.csv")
    # Porzucenie niepotrzebnych kolumn
    pit_stops=pit_stops.drop(["time","duration","milliseconds"],axis=1)
    # Połączenie danych o czasach okrążeń z danymi o pit stopach (z tego
    # będzie można ustalić czy dane okrążenie było zakończone zjazdem do boksu)
    data=lap_times.merge(pit_stops,on=["raceId","driverId","lap"],how="left")
    # Na tym etapie, możemy ręcznie narzucić pewne wymogi co do danych,
    # dzięki "niezmienności" pewnych kwestii w F1
    # Np. Najszybsze okrążenie w historii F1 to trochę ok. 53s , więc
    # usunę okrążenia, które trwały mniej niż 50s (50000 milisekund) oraz te, które trwały więcej niż 140s (120000 milisekund)
    data=data[(data["milliseconds"]>50000)]
    data=data[(data["milliseconds"]<140000)]

    # W miejscu NaN w kolumnie stop (NaN powstały w wyniku operacji merge),
    # wstawiamy 0 (oznacza to, że kierowca nie zjeżdżał do boksu),
    # każda inna wartość oznacza zjazd do boksu
    data["stop"]=data["stop"].fillna(0)
    data["stop"]=data["stop"].apply(lambda x: 1 if x!=0 else 0)
    data=data.rename(columns={"stop":"isPitStop"})
    
    # Ponowna operacja merge z danymi odnośnie pit stopów, by dodać kolumnę
    # is_out_lap, która będzie informować czy dane okrążenie było okrążeniem wyjazdowym
    # z boksu (kierowca opuszcza boks)
    pit_stops=pit_stops.drop("stop",axis=1)
    data=data.merge(pit_stops,on=["raceId","driverId"],how="left")
    data.rename(columns={"lap_x":"currentLap","lap_y":"boxLap"},inplace=True)
    # Jeżeli NaN, kierowca nie zjechał już do końca trwania wyścigu
    # (albo go nie ukończył albo nie zjeżdżał do boksu)
    data["boxLap"]=data["boxLap"].fillna(0)
    data=data.sort_values("boxLap",ascending=False).drop_duplicates(subset=["raceId","driverId","currentLap","milliseconds"],keep="first")
    data=data.reset_index(drop=True)
    # Initializacja kolumny isOutLap
    data["isOutLap"]=0
    data["isOutLap"] = data.apply(lambda row: 1 if row["currentLap"] + 1 == row["boxLap"] else 0, axis=1)
    data.drop("boxLap",axis=1,inplace=True)
    
    # Operacja merge z danymi odnośnie końcowych rezultatów
    results=pd.read_csv("data/results.csv")
    # Porzucenie niepotrzebnych kolumn
    results=results[["raceId","driverId","constructorId","grid"]]
    data=data.merge(results,on=["raceId","driverId"],how="left")
    
    # Operacja merge z danymi odnośnie kwalifikacji
    quali=pd.read_csv("data/qualifying.csv")
    quali=quali[["raceId","driverId","q1","q2","q3"]]
    # Zmiana czasu z formatu X:XX.XXXX na czas w minutach
    quali["q1"]=quali["q1"].apply(lambda x: convert_time_to_min(x))
    quali["q2"]=quali["q2"].apply(lambda x: convert_time_to_min(x))
    quali["q3"]=quali["q3"].apply(lambda x: convert_time_to_min(x))
    # Obliczenie najlepszego czasu kwalifikacji (najmniejszy czas z q1,q2,q3 lub jedyny lub 0)
    quali["bestQualiTime"]=quali[["q1","q2","q3"]].min(axis=1)
    quali["bestQualiTime"] = quali[["q1", "q2", "q3"]].min(axis=1,skipna=True)
    # Porzucenie niepotrzebnych kolumn
    quali=quali.drop(["q1","q2","q3"],axis=1)
    data=data.merge(quali,on=["raceId","driverId"],how="left")
    data["bestQualiTime"] = data["bestQualiTime"].astype(float).fillna(0)
    
    # Odrzucenie rzędów z brakującymi danymi odnośnie czasu kwalifikacji
    data=data[data["bestQualiTime"]!=0]
    
    # Operacja merge z danymi odnośnie toru
    races=pd.read_csv("data/races.csv")
    races=races[["raceId","circuitId"]]
    data=data.merge(races,on="raceId",how="left")

    data["time"]=data["milliseconds"].apply(lambda x: x/60000)
    
    # Odrzucenie obserwacji odstających
    # W F1 czas pierwszego okrążenia zawsze jest wolniejszy od reszty 
    # (kierowcy startują z miejsca)
    
    data=data[data["currentLap"]!=1]
    return data

data=read_data()

In [207]:
 # Zmiana kolejności kolumn dla czytelności
data=data[[
        "raceId",
        "circuitId",
        "driverId",
        "constructorId",
        "grid",
        "currentLap",
        "position",
        "isPitStop",
        "isOutLap",
        "bestQualiTime",
        "time"]]

In [208]:
print(data[:5])

   raceId  circuitId  driverId  constructorId  grid  currentLap  position  \
0    1046          3       847            131     2           3         1   
2    1046          3       847            131     2           2         1   
3    1046          3       847            131     2           7         1   
4    1046          3       847            131     2           6         1   
5    1046          3       847            131     2          52         1   

   isPitStop  isOutLap  bestQualiTime      time  
0          0         0        0.89005  1.497167  
2          0         0        0.89005  1.640350  
3          0         0        0.89005  0.969333  
4          0         0        0.89005  1.478300  
5          0         0        0.89005  0.951867  


In [209]:
print("Ilość obserwacji:",data.shape[0])

Ilość obserwacji: 464089


### 2. Dalszy preprocessing danych

In [210]:
from sklearn.model_selection import train_test_split

train_data,test_data=train_test_split(data,test_size=0.2,shuffle=True)

Skalowanie danych które zostaną wykorzystane w niektórych modelach. Zastosowałem skalowanie tylko dla kolumn związanych z czasem przejazdu.

In [211]:
from sklearn.preprocessing import StandardScaler

columns_names_to_scale=["bestQualiTime","time"]
scaler=StandardScaler()
columns_to_scale=train_data[columns_names_to_scale]
scaler.fit(columns_to_scale)
train_data_scaled=train_data.copy()
test_data_scaled=test_data.copy()
train_data_scaled[columns_names_to_scale]=scaler.transform(columns_to_scale)
test_data_scaled[columns_names_to_scale]=scaler.transform(test_data[columns_names_to_scale])


"Wyciągnięcie" jednej obserwacji w celu prezentacji.

In [212]:
index_to_pick=859
example=train_data.loc[[859]]
train_data=train_data.drop(859)
print(example)
example_X=example.drop(["time"],axis=1)
example_y=example["time"]

     raceId  circuitId  driverId  constructorId  grid  currentLap  position  \
859    1107         70       830              9     1          33         2   

     isPitStop  isOutLap  bestQualiTime      time  
859          0         0        1.07318  1.160517  


In [213]:
# Podział na "X" i "y" danych bez skalowania i ze skalowaniem.

X_train=train_data.drop("time",axis=1)
y_train=train_data["time"]
X_test=test_data.drop("time",axis=1)
y_test=test_data["time"]

X_train_scaled=train_data_scaled.drop("time",axis=1)
y_train_scaled=train_data_scaled["time"]
X_test_scaled=test_data_scaled.drop("time",axis=1)
y_test_scaled=test_data_scaled["time"]

Funkcja wyświetlająca w "ładniejszy" sposób wartości metryk RMSE oraz MAE dla modeli wykorzystywanych poniżej.

In [214]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import median_absolute_error
def print_metrics(y_true,y_pred,model_name=None):
    mse=root_mean_squared_error(y_true,y_pred)
    mae=median_absolute_error(y_true,y_pred)
    print(f"Model: {model_name}\nRMSE: {mse:.5f}\nMAE score: {mae:.5f}")

Funkcja wyświetlająca czas rzeczywisty i przewidywany dla 1 przykładu 

In [236]:
def print_example(model):
    prediction=model.predict(example_X)
    print(f"\nResult for example\nReal time: {example_y.values[0]}, predicted time: {prediction[0]}")

### 3. Modele

#### 1. Model zwykłej regresji liniowej.

In [237]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print_metrics(y_test,y_pred,model_name="Linear Regression")

print_example(model)

Model: Linear Regression
RMSE: 0.14161
MAE score: 0.04616

Result for example
Real time: 1.1605166666666666, predicted time: 1.2237933111506445


#### 2. Model regresji liniowej z przeskalowanymi danymi dotyczących czasu.

In [238]:
model=LinearRegression()
model.fit(X_train_scaled,y_train_scaled)
y_pred=model.predict(X_test_scaled)
print_metrics(y_test_scaled,y_pred,model_name="Linear Regression bestQualiTime and time scaled with StandardScaler")

print_example(model)

Model: Linear Regression bestQualiTime and time scaled with StandardScaler
RMSE: 0.62443
MAE score: 0.20353

Result for example
Real time: 1.1605166666666666, predicted time: 0.7665661040357635


#### 3. Model z regularyzacją z wykorzystaniem klasy Ridge.

In [239]:
from sklearn.linear_model import Ridge
# Wartość alpha została podana po prostu jako przykład
model=Ridge(alpha=3.3)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print_metrics(y_test,y_pred,model_name="Ridge with lambda=3.3")

print_example(model)

Model: Ridge with lambda=3.3
RMSE: 0.14161
MAE score: 0.04618

Result for example
Real time: 1.1605166666666666, predicted time: 1.223861427773873


#### 4. Model z regularyzacją z wykorzystaniem Lasso.

In [240]:
from sklearn.linear_model import Lasso
# Wartość alpha została podana po prostu jako przykład
model=Lasso(alpha=0.1)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print_metrics(y_test,y_pred,model_name="Lasso with lambda=0.1")

print_example(model)

Model: Lasso with lambda=0.1
RMSE: 0.21861
MAE score: 0.15625

Result for example
Real time: 1.1605166666666666, predicted time: 1.5406024100748354


#### 5. Model lasu losowego dla problemu regresji.

In [241]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_estimators=50,max_depth=30)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print_metrics(y_test,y_pred,model_name="Random Forest with 50 estimators and max depth 30")

print_example(model)

Model: Random Forest with 50 estimators and max depth 30
RMSE: 0.07256
MAE score: 0.00802

Result for example
Real time: 1.1605166666666666, predicted time: 1.161123666666667


#### 6. Model sieci neuronowej w PyTorch'u

**Budowa sieci**:
- warstwa droput z domyślnym prawdopodobieństwem wyzerowania 30% (wraz z eksperymentami zmniejszyłem ją do 10%)
- warstwa liniowa o wymiarach: rozmiaru danych wejściowych na wejściu tej warstwy i 64 na wyjściu
- warstwa liniowa o wymiarach: 64 na wejściu, 32 na wyjściu
- warstwa liniowa o wymiarach: 32 na wejśćiu, 1 na wyjściu\
Funkcje aktywacji 2 pierwszych warstw liniowych to *ReLU*.\
Wykorzystywany optymalizator to *Adam*.\
Funkcją błędu jest *RMSE*.\
Zastosowane zostały również następujące parametry:
- rozmiar batcha to **64**
- współczynnik uczenia to **0.001** (testowałem różne wartości, ta uzyskiwała najlepsze wyniki)

In [242]:
import torch

X_train_tensor=torch.tensor(X_train.values).float()
y_train_tensor=torch.tensor(y_train.values).float()

X_test_tensor=torch.tensor(X_test.values).float()
y_test_tensor=torch.tensor(y_test.values).float()

In [243]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Model sieci neuronowej

class F1LapTimePredictionModel(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.3):
        super(F1LapTimePredictionModel, self).__init__()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        output = self.fc3(x)
        return output

In [244]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Parametry trenowania

learning_rate = 0.001
epochs = 20
batch_size = 64
log_interval = 5000  

input_dim = X_train_tensor.shape[1]
model = F1LapTimePredictionModel(input_dim=input_dim, dropout_rate=0.3)
model.to(device)

# Z powodu braku implementacji RMSE w PyTorch, musiałem zastosować pewien "trick"
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.train()
for epoch in range(epochs):
    epoch_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        predictions = model(data)
        # Tutaj wspomniany wcześniej "trick", obliczam RMSE z MSE, RMSE = sqrt(MSE)
        loss = torch.sqrt(criterion(predictions.squeeze(), target))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        if batch_idx % log_interval == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.5f}")
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"End of epoch {epoch+1}/{epochs}, Loss: {avg_epoch_loss:.5f}")


Using device: cuda
Epoch 1/20, Loss: 5.48503
Epoch 1/20, Loss: 0.45204
End of epoch 1/20, Loss: 0.39097
Epoch 2/20, Loss: 0.30533
Epoch 2/20, Loss: 0.23990
End of epoch 2/20, Loss: 0.26069
Epoch 3/20, Loss: 0.28529
Epoch 3/20, Loss: 0.24109
End of epoch 3/20, Loss: 0.23501
Epoch 4/20, Loss: 0.27980
Epoch 4/20, Loss: 0.22151
End of epoch 4/20, Loss: 0.21578
Epoch 5/20, Loss: 0.23460
Epoch 5/20, Loss: 0.18890
End of epoch 5/20, Loss: 0.20089
Epoch 6/20, Loss: 0.21524
Epoch 6/20, Loss: 0.19374
End of epoch 6/20, Loss: 0.18770
Epoch 7/20, Loss: 0.17945
Epoch 7/20, Loss: 0.19859
End of epoch 7/20, Loss: 0.18137
Epoch 8/20, Loss: 0.11339
Epoch 8/20, Loss: 0.18783
End of epoch 8/20, Loss: 0.17898
Epoch 9/20, Loss: 0.17368
Epoch 9/20, Loss: 0.16277
End of epoch 9/20, Loss: 0.17741
Epoch 10/20, Loss: 0.16417
Epoch 10/20, Loss: 0.17227
End of epoch 10/20, Loss: 0.17602
Epoch 11/20, Loss: 0.13541
Epoch 11/20, Loss: 0.18049
End of epoch 11/20, Loss: 0.17499
Epoch 12/20, Loss: 0.14092
Epoch 12/20, 

#### Ewaluacja modelu sieci neuronowej

In [245]:
test_loss_rmse = 0.0
test_loss_mae = 0.0

model.eval()
mae_loss = nn.L1Loss()

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)

        test_predictions = model(data)
        mse_loss = criterion(test_predictions.squeeze(), target)
        rmse_loss = torch.sqrt(mse_loss)
        test_loss_rmse += rmse_loss.item()

        mae = mae_loss(test_predictions.squeeze(), target)
        test_loss_mae += mae.item()

avg_test_loss_rmse = test_loss_rmse / len(test_loader)
avg_test_loss_mae = test_loss_mae / len(test_loader)

print(f"Test Loss RMSE: {avg_test_loss_rmse:.4f}, MAE Score: {avg_test_loss_mae:.4f}")

print("Example prediction")
example_X_tensor = torch.tensor(example_X.values).float()
example_y_tensor = torch.tensor(example_y.values).float()
example_X_tensor = example_X_tensor.to(device)
example_y_tensor = example_y_tensor.to(device)
example_prediction = model(example_X_tensor)
print(f"Real time: {example_y.values[0]}, predicted time: {example_prediction.item()}")

Test Loss RMSE: 0.2920, MAE Score: 0.2362
Example prediction
Real time: 1.1605166666666666, predicted time: 1.2433815002441406
